### Program flow
 * Read data 
     Testfolders has subfolders 01 and 02 containing .tif
     Trainingfolders has subfolders 01 01_GT 01_ST 02 02_GT 02_ST
     GT has subfolders SEG and TRA ST has subfolder SEG
     SEG contains .tif that 

### Function definitions

In [1]:
%run helpers.ipynb

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from skimage.segmentation import flood, flood_fill, find_boundaries
from scipy.spatial import distance_matrix
from scipy.ndimage.morphology import distance_transform_edt
from tqdm import tqdm


def read_data(folder):
    X = []
    Y = []
    path = os.fsencode(folder)
    for folder in os.listdir(path):
        if not folder.startswith(b'.') and not folder.endswith(b'GT'):
            inner_path = concat_path(path, folder)
            for file in os.listdir(inner_path):
                if not file.startswith(b'.'):
                    file_path = concat_path(inner_path, file)
                    im = np.array(Image.open(file_path))
                    foo = np.expand_dims(im, axis = 2)
                    file_id = concat_path(folder, file)
                    point = (file_id, im)
                    Y.append(point) if folder.endswith(b'ST') else X.append(point)
    return X, Y


def fetch_data(folder):
    path = os.getcwd()+'/' + folder
    test_folder = path+'-test'
    training_folder = path+'-training'
    X_test, _ = read_data(test_folder)
    X_train, Y_train = read_data(training_folder)
    X_test.sort()
    X_train.sort()
    Y_train.sort()
    
    X_train = list(map(lambda x : x[1], X_train))
    Y_train = list(map(lambda y : y[1], Y_train))
    X_test = list(map(lambda x : x[1], X_test))
    return X_test, X_train, Y_train


def is_border(a,b):
    return (a != 0  and a != b)
    
    
def enforce_borders(labels):
    for i in range(1,len(labels)-1):
        for j in range(1,len(labels[0])-1):
            curr   = labels[i][j]
            over   = labels[i][j-1] 
            prev   = labels[i-1][j]
            diag_1 = labels[i-1][j-1] 
            diag_2 = labels[i+1][j-1] 
            if(is_border(over,curr) or is_border(prev,curr) or is_border(diag_1,curr) or is_border(diag_2,curr)):
                labels[i][j] = 0
                
    return labels


def machine_boundries(labels):
    a = find_boundaries(labels, mode='thick', connectivity=4)
    for i in range(512):
        for j in range(512):
            if a[i,j] == 1:
                
                labels[i,j] = 0
    return labels


def reinforce_borders(masks):
    amount = len(masks)
    reinforced = np.zeros((amount, 512, 512))
    print('Reinforcing borders of ', amount, 'images')
    for i in tqdm(range(amount)):
        enforced = enforce_borders(masks[i])
        enhanced = machine_boundries(enforced)
        reinforced[i] = enhanced
    return reinforced


def unify_all_images(masks):
    amount = len(masks)
    unified = np.zeros((amount, 512, 512, 1))
    print('Unifying classes of ', amount, 'masks')
    for i in tqdm(range(amount)):
        unified[i] = np.minimum(masks[i], 1)
    return unified


def fill_classes(img):
    img = squeeze_image(img)
    label = 2
    for i in range(len(img)):
        for j in range(len(img[0])):
            if(img[i][j] == 1):
                img = flood_fill(img, (i, j), label)
                label += 1
    return img


def adjust_dimension_to_network(data):
    return np.expand_dims(data, axis = 3)


def display_image(img):
    img = squeeze_image(img)
    imgplot = plt.imshow(img)    
    plt.colorbar()
    plt.show()

    
def squeeze_image(img):
    img = np.squeeze(img) if len(img.shape) == 4 else img
    img = np.squeeze(img) if len(img.shape) == 3 else img
    return img
    
    
def divide_data(data, split):
    samples = len(data)
    index = int(samples*split)
    test = data[index:samples]
    train = data[0:index]
    return train, test
    
    
folder_1 = 'datasets/DIC-C2DH-HeLa'
folder_2 = 'datasets/PhC-C2DH-U373'


def dist_to_n_nearest(img, x, y, distance, summ):
    not_zeros = np.argwhere(img != 0)
    dist_matrix = distance_matrix([(x, y)], not_zeros, p=2)
    min_distance = np.min(dist_matrix)
    min_index = np.argmin(dist_matrix)
    min_point = not_zeros[np.argmin(dist_matrix)]
    min_x, min_y = min_point[0], min_point[1]
    img = remove_cell(img, min_x, min_y)
    if distance == 1: 
        return min_distance + summ
    else:
        return dist_to_n_nearest(img, x, y, distance-1, min_distance)
    
    
def fill_dist_matrix(img):
    dist_matrix = np.zeros_like(img)
    for i in tqdm(range(len(img))):
        for j in range(len(img[0])):
            dist_matrix[i, j] = dist_to_n_nearest(np.copy(img), i, j, 2, 0)
    return dist_matrix
    
    
def fill_all_dist_matrices(labels):
    dist_matrices = np.zeros_like(labels)
    for i, label in tqdm(enumerate(labels)):
        dist_matrices[i] = fill_dist_matrix(label)
    return dist_matrices

    
def remove_cell(img, x, y):
    return flood_fill(img, (x, y), 0)


def fetch_border_weights(Y):
    Y = squeeze_image(np.around(Y))
    colors = np.unique(Y)
    zero_index = np.argwhere(colors == 0)
    colors = np.delete(colors, zero_index)
    distance_images = np.ones((len(colors),512,512))
    for index, color in enumerate(colors):
        ones = np.argwhere(Y == color)
        x, y = ones[0]
        a = np.ones((512,512))
        for i in ones:
            a[i[0], i[1]] = 0
            #display_image(a)
        Y1 = distance_transform_edt(a)
            #display_image(Y1)
        distance_images[index] = Y1

    distance_images = np.transpose(distance_images)
    distance_images = np.sort(distance_images, axis=2)
    distance_images = distance_images[:,:,0:2]
    weights = np.sum(distance_images, axis=2)
    return np.transpose(weights)


def fetch_weights(masks):
    print('Generating weight matrices for: ', len(masks), ' masks.')
    weights = np.zeros((len(masks), 512, 512))
    unified_masks = np.array(unify_all_images(masks))
    class_weights = np.zeros((512, 512))
    class_weight_scale = 512 * 512 * (len(masks)/8)
    w1 = 1 - unified_masks.sum() / class_weight_scale
    w2 = 1 - w1
    avg = np.squeeze(unified_masks.sum(0))
    class_weights[avg == 1] = w1
    class_weights[avg == 0] = w2
    sigma = 25
    scale = 2 * (sigma ** 2)
    w0 = 10
     
    for i, mask in tqdm(enumerate(masks)):
        border_weights = fetch_border_weights(mask)
        scaled_border_weights = class_weights + w0 * np.exp(-(np.power(border_weights, 2))/scale)
        combined_weights = class_weights + scaled_border_weights
        weights[i] = combined_weights
    return weights

### Run this to load data and reinforce borders
Unifying of labels is done later due to the nature of the weight calculations

In [3]:
X_test, X_train, Y_train = fetch_data(folder_1)
#Y_train = reinforce_borders(Y_orig)

Y_train = adjust_dimension_to_network(Y_train)
X_train = adjust_dimension_to_network(X_train)
X_test = adjust_dimension_to_network(X_test)
#Y_train = unify_all_images(Y_train)

### Run this to create augmentations and calculate weights

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_args = dict(rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest')

seed = 1

datagen_X = ImageDataGenerator(**datagen_args)
datagen_Y = ImageDataGenerator(**datagen_args)
  
datagen_X.fit(X_train, augment=True, seed=seed)
datagen_Y.fit(Y_train, augment=True, seed=seed)

amount_of_images = len(X_train)
augmentations = 3
amount_of_new_images = amount_of_images * augmentations
X_new = np.ones((amount_of_new_images, 512, 512, 1))
Y_new = np.ones((amount_of_new_images, 512, 512, 1))


print('Creating ', amount_of_new_images, ' augmentations of ', amount_of_images, ' images')
for j in tqdm(range(amount_of_images)):
    i = 0
    for x_batch in datagen_X.flow(X_train[j:j+1], batch_size=1, seed=seed):
        if(i >= augmentations):
            break
        save_index = j * augmentations + i
        X_new[save_index] = np.around(x_batch)
        #display_image(x_batch)
        i +=1
        
for j in tqdm(range(amount_of_images)):
    i = 0
    for y_batch in datagen_Y.flow(Y_train[j:j+1], batch_size=1, seed=seed):
        if(i >= augmentations):
            break
        save_index = j * augmentations + i
        Y_new[save_index] = np.around(y_batch)
        #display_image(y_batch)
        i +=1
        
import pickle
#pickle_out = open("X_with_augmentations.pkl","wb")
#pickle.dump(X_new, pickle_out)        

#weights = fetch_weights(Y_new)

Y_new_unified = unify_all_images(Y_new)

#pickle_out = open("Y_with_augmentations.pkl","wb")
#pickle.dump(Y_new_unified, pickle_out)        

#pickle_out = open("weights.pkl","wb")
#pickle.dump(weights, pickle_out)    

  1%|▍                                                                                 | 1/168 [00:00<00:17,  9.70it/s]

Creating  504  augmentations of  168  images


 16%|█████████████                                                                   | 82/504 [00:00<00:01, 407.55it/s]

Unifying classes of  504 masks


100%|███████████████████████████████████████████████████████████████████████████████| 504/504 [00:01<00:00, 397.72it/s]


In [ ]:
import tensorflow as tf
from tensorflow.python.framework import smart_cond
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
import numpy as np

IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

def conv2D_layer(filters):
    return tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', kernel_initializer = 'he_normal', padding = 'same')

inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

s = tf.keras.layers.Lambda(lambda x : x / 255)(inputs)

#Contraction path
c1 = conv2D_layer(64)(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = conv2D_layer(64)(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = conv2D_layer(128)(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = conv2D_layer(128)(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = conv2D_layer(256)(p2)
c3 = tf.keras.layers.Dropout(0.1)(c3)
c3 = conv2D_layer(256)(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = conv2D_layer(512)(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = conv2D_layer(512)(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = conv2D_layer(1024)(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = conv2D_layer(1024)(c5)

#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides = (2,2), padding = 'same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = conv2D_layer(512)(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = conv2D_layer(512)(c6)

u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides = (2,2), padding = 'same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = conv2D_layer(256)(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = conv2D_layer(256)(c7)

u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides = (2,2), padding = 'same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = conv2D_layer(128)(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = conv2D_layer(128)(c8)

u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides = (2,2), padding = 'same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = conv2D_layer(64)(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = conv2D_layer(64)(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation = 'sigmoid')(c9)

checkpointer = tf.keras.callbacks.ModelCheckpoint('backup_model.h5', verbose = 1, save_best_only = True)

callbacks = [ 
    tf.keras.callbacks.EarlyStopping(patience = 4, monitor = 'loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]


def cast_y(y_true, y_pred):
    y_pred = ops.convert_to_tensor_v2(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    return y_true, y_pred


def transformer(x):
    x.numpy()
    return x


def balanced_cross_entropy(beta):
    def convert_to_logits(y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        return tf.math.log(y_pred / (1 - y_pred))

    def loss(y_true, y_pred):
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = math_ops.cast(y_true, y_pred.dtype)
        
        y_pred = convert_to_logits(y_pred)
        pos_weight = beta / (1 - beta)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, labels=y_true, pos_weight=pos_weight)
        return tf.reduce_mean(loss * (1 - beta))
    return loss
    

def energy_loss(y_true, y_pred):
    #bca = balanced_cross_entropy(0.8)(y_true, y_pred)
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

_epsilon = tf.convert_to_tensor(K.epsilon(), np.float32)

c11 = tf.keras.layers.Lambda(lambda x: x / tf.reduce_sum(x, len(x.get_shape()) - 1, True))(outputs)
c11 = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(x, _epsilon, 1. - _epsilon))(c11)
c11 = tf.keras.layers.Lambda(lambda x: K.log(x))(c11)
weight_ip = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
weighted_sm = tf.keras.layers.multiply([c11, weight_ip])

def my_loss(target, output):
        return tf.reduce_sum(target * output, len(output.get_shape()) - 1)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

sgd = tf.keras.optimizers.SGD(lr=1e-7, decay=1e-6, momentum=0.99, nesterov=True)
#model = tf.keras.Model(inputs = [inputs, weight_ip], outputs = [weighted_sm])
model = tf.keras.Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = sgd , loss = 'binary_crossentropy', metrics=[get_f1])
#model.summary()

#pickle_out = open("X_with_augmentations.pkl","wb")
#X_new = pickle.load(X_new, pickle_out)        

#pickle_out = open("Y_with_augmentations.pkl","wb")
#Y_new = pickle.load(Y_new_unified, pickle_out)        

#pickle_out = open("weights.pkl","wb")
#weights = pickle.load(weights, pickle_out)    

#results = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=1), epochs=25, callbacks = callbacks)
#X_white = np.ones((lenght, 512, 512, 1))
results = model.fit(X_new, Y_new_unified, validation_split = 0.1, batch_size = 1, epochs = 25, callbacks = callbacks)
#results = model.fit((X_new, adjust_dimension_to_network(weights)), Y_new_unified, validation_split = 0.1, batch_size = 1, epochs = 25, callbacks = callbacks)




Train on 453 samples, validate on 51 samples
Epoch 1/25
453/453 [==============================] - 112s 248ms/sample - loss: 0.7101 - get_f1: 0.2715 - val_loss: 0.7107 - val_get_f1: 0.0122
Epoch 2/25
453/453 [==============================] - 111s 245ms/sample - loss: 0.7083 - get_f1: 0.2995 - val_loss: 0.7078 - val_get_f1: 0.0288
Epoch 3/25
453/453 [==============================] - 111s 244ms/sample - loss: 0.7067 - get_f1: 0.3254 - val_loss: 0.7055 - val_get_f1: 0.0655
Epoch 4/25
453/453 [==============================] - 111s 244ms/sample - loss: 0.7053 - get_f1: 0.3492 - val_loss: 0.7035 - val_get_f1: 0.1268
Epoch 5/25
453/453 [==============================] - 111s 245ms/sample - loss: 0.7043 - get_f1: 0.3704 - val_loss: 0.7017 - val_get_f1: 0.1926
Epoch 6/25
453/453 [==============================] - 111s 244ms/sample - loss: 0.7034 - get_f1: 0.3905 - val_loss: 0.7001 - val_get_f1: 0.2571
Epoch 7/25
453/453 [==============================] - 111s 244ms/sample - loss: 0.7027 - ge

In [ ]:
foo = Y_new_unified[0]
colors = np.unique(foo)

display_image(X_new[0])
display_image(np.around(X_new[0]))




colorsX = np.unique(np.around(X_new[0]))
print(len(colors))
for i in range(60):
    print(np.unique(Y_new_unified[i]))
    print(len(np.unique(Y_new_unified[i])))

-----------------------------------------------------------

In [ ]:
display_image(weights[0])
display_image(Y_new_unified[0])
im = np.expand_dims(X_new[0], axis=3)

z = np.ones((1, 512, 512, 1))
print("hehehe"+str(weights[0:1].shape))
predictions = model.predict([X_new[0:1], z])
#predictions = model.predict(X_train)


display_image(X_new[0])
#print(X_train[0])
foo = np.squeeze(predictions[0])
display_image(foo)
border = 0.55
foo[foo < border] = 0
foo[foo >= border] = 123
print(foo)
display_image(foo)
print(len(predictions[0].shape))

foo = foo ** 10
